In [2]:

import pandas as pd
import sys,os,inspect
import calendar
from datetime import datetime
from datetime import timedelta
from cronsim import CronSim
import argparse
import json
import PySimpleGUI as sg
#import PySimpleGUIWeb as sg
import pathlib
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import textwrap
import re
from shlex import split
import webbrowser
import subprocess
import collections
import pyglet,tkinter
from pyglet import font
# import OpenGL
# from OpenGL import GLU
font.add_file('/etc/fonts/fonts/CENTAUR.TTF')
font='Courier 10 bold '
bicHome = "/home/joe/bic_etl/"
numWindows=0
headerStore = {}

colorPairs = [["#D6EAF8","#85C1E9"],["#b3f0ff","#33d6ff"],["#D5F5E3","#A3E4D7"],["#FCF3CF","#F7DC6F"]]
windowsOpen = {}
windowsOpen["main"] = []
windowsOpen["unique"] = []

#############################################
def compareWindow(stats1,df1,file1):
    global color1,color2
    header_list = ["Column","% Missing","Missing","string","integer","float","boolean"]
    print("IN COMP")
    col_widths = [8]*len(header_list)
    col_widths[0] = 25
    columnSortStateTable1 = {}
  

    ## set up sort state for the column in both tables
    for col in header_list:
         columnSortStateTable1[col] = -1
       
            
    valsFile1 = getValues(stats1,header_list)
   
    
    rowFile1Colors = setRowColors(valsFile1,color1,color2,"pink",header_list)
    
    layCol1 = [[sg.Text(f"File 1 {file1}",font="CENTAUR 15")],[sg.Text(f"File 1 Shape {df1.shape}",font="CENTAUR 15")],
               [sg.Table(values=valsFile1,text_color="black", auto_size_columns=False,enable_events=True,num_rows=20,col_widths=col_widths,font="CENTAUR 10",
                   justification='center',pad=(5,5),vertical_scroll_only=False,
                   key='-TABLEFILE-',row_colors=rowFile1Colors,headings = header_list,metadata=columnSortStateTable1)]
               ]
    

    layout = [[sg.Button("Quit")],
    layCol1]  
        

              
              
    window2 = sg.Window('Compare',layout,finalize=True,resizable=True)
    a = window2.CurrentLocation()
    screen_width, screen_height = window2.get_screen_dimensions()
    win_width, win_height = window2.size
    x, y = (screen_width - win_width)//2, (screen_height - win_height)//2
    x=200
    y=200
    window2.move(x, y)
    tableFile1 = window2['-TABLEFILE-']
    tableFile1.bind('<Button-1>', "Click")
    headerStore[tableFile1] = header_list
    return window2,tableFile1,valsFile1,rowFile1Colors




#######################################################

def stringArgs(string):
#  This is set to decode the extract string to get the location and name of the 
#  extracted data file
    h =split(string)
    inf = h.index("-f")
    inf = h[inf+1]
    ot = h.index("-a")
    ot = h[ot+1]
    of = h.index("-o")
    of = h[of+1]
    inf=inf.replace(inf[-4:],ot)
    f = inf.split("/")
    finalFile = f"{bicHome}{of}{f[-1]}"
    return finalFile
 
###################################################    

def splitL(data):
    if data:
        head, *tail = data  # This is a nicer way of doing head, tail = data[0], data[1:]
        return {head: splitL(tail)}
    else:
        return []



###############################################################################

def go_deeper(aDict,value,nit):
    nit+=1
    for k, v in aDict.items():
         if not bool(v):            
             aDict[k] = []
             aDict[k].append(value)
         elif isinstance(v,list):
             aDict[k].append(value)
         else:
             go_deeper(v,value,nit)
   
    return aDict

def init():
    global dataSetsEtl
    groups = []
    desktop = pathlib.Path("/home/joe/bic_etl")
    runEtls = []
    dataSets = []
    # .rglob() produces a generator too
    desktop.rglob("*")
    files = list(desktop.rglob("*"))
# Which you can wrap in a list() constructor to materialize
    for ff in files:

            if (str(ff).split("/")[-1] == "run_etl.json"):     
                a=str(ff).split("/")
                m = a.index("bic_etl")
                b = a[m+1:-1]
                ll = splitL(b)
                groups.append(ll)
                runEtls.append(ff)
            
    print(f"{len(runEtls)} run_etl.json files found")    
    
    dataSets = []
    groups = []
    for file in runEtls:
      f = open(file,"r")
      data = json.load(f)
      # print(file)
      # print("----")
      a=str(file).split("/")
      m = a.index("bic_etl")
      b = a[m+1:-1]
      ll = splitL(b)
    #  print(ll)
    #  groups.append(ll)
      for val in data:
            if "title" in val:
                  title=val["title"]
    #              print(title,ll)
                  nit=0
                  ll = go_deeper(ll,title,nit)
     #             print(ll)
     #             groups.append(ll)
            dataSets.append(val)
    #  print("FF ",ll) 
      groups.append(ll)

    datasets = []
    dataSetsEtl={}
    for val in dataSets:
        if "title" in val:
          datasets.append(val["title"])
          title=val["title"]
          dataSetsEtl[title] = {}  
          for k,v in val.items():
          #      print(k,v)
                if k != "title":
                    dataSetsEtl[title][k] = {}
                    if isinstance(v,dict):
                        for k1,v1 in v.items():
                            dataSetsEtl[title][k][k1]=v1

    return sorted(datasets)

def map4x4(row):
    
    if isinstance(row["Data Link"],str) and  len(row["Data Link"]) == 9 and re.findall( "\w{4}-\w{4}",row["Data Link"]):
  #      link = '<a href="https://data.colorado.gov/dataset/{}">{}</a>'.format(row["Data Link"],row["Data Link"])
        
        link = 'https://data.colorado.gov/dataset/{}'.format(row["Data Link"])
    else:
        link = ""
        
    return link
    
#     mf["CIM Link"] = mf.apply(map4x4,axis=1)

#     mf=mf.loc[~mf["Standardized Title for Dataset"].isna()]
#     mfShort = mf[['Standardized Title for Dataset', 'Data Link','CIM Data Type','GoCodePublishYear','Standardized Short Description','CIM Link']]
#     mfShort["Standardized Title for Dataset"] = mfShort["Standardized Title for Dataset"].astype(str)

#####################################################################

def exceptionLog(exception,funCall):
  exception_message = str(exception)
  exception_type, exception_object, exception_traceback = sys.exc_info()
  filename = os.path.split(exception_traceback.tb_frame.f_code.co_filename)[1]
  print(f"{exception_message} {exception_type} {funCall}, Line {exception_traceback.tb_lineno}")

######################################################    
    
def getFile(how,file,WindowP):
    if how == "Local":
        if file[-3:].lower() == "tsv":
            delim = "\t"
            df=pd.read_csv(file,encoding="latin",delimiter=delim)
        elif file[-4:].lower() == "xlsx":
            df=pd.read_excel(file,engine="openpyxl")
        else:
            try:
                df=pd.read_csv(file)
            except Exception as err:
                print("Error, trying with encoding=latin")
                df=pd.read_csv(file,encoding="latin")
    elif how == "Fetch":
       # print("Getting that web data")
        file=values["-WEB-"]
        getPrevFiles(2,file)

  #      windowP["-PINFO-"].update(f"START reading WEB File:{file}:")
        df=pd.read_csv(file)
  #      windowP["-PINFO-"].update(f"FINISHED reading WEB File:{file}:")
        
    return df

##############################################################
def setRowColors(lst,col1,col2,colsp,header):
    count=0
    colors = {}
    nrec = header.index("% Missing")
    for vals in lst:
        key = vals[0]
        if count%2 == 0:
            colors[key] = col1
        else:
            colors[key] = col2
        if vals[nrec] > 99.0:
            colors[key] = colsp
        count+=1
    colTab = []
    for key,colr in colors.items():
        colTab.append(colr)
    rowNums = [num for num in range(0,len(colTab)+1)]
    colText = ["black"]*len(colTab)
    colrw = list(zip(rowNums,colTab))
    print("SETTING COLS ",col1,col2)
    return colrw

###########################################################
def sortTable(row,stats,table,event,header):
    
        e = table.user_bind_event 
        region = table.Widget.identify('region', e.x, e.y)
        if region == 'heading':
            row = 0
        elif region == 'cell':
            row = int(table.Widget.identify_row(e.y))
   
        if row == 0:
            colSortState = table.metadata
            colClicked = int(table.Widget.identify_column(e.x)[1:])
            header = headerStore[table]
            statClicked = header[colClicked-1].strip()
            print("clicked ",colClicked,statClicked,header)
            colSortState[statClicked]*=-1
            if colSortState[statClicked] == -1:
                sortAsc=False
            else:
                sortAsc=True
            if colClicked > 1:  # user number sort
                statsS = dict(sorted(stats.items(), key=lambda x: x[1][statClicked],reverse=sortAsc))
            else:
                statsS = dict(sorted(stats.items(), key=lambda x: x[0],reverse=sortAsc))


            statsVals=[]
            for col in statsS:
                 vals=[]
                 vals.append(col)
                 for k in header[1:]:
                    vals.append(statsS[col][k.strip()])
                 statsVals.append(vals)
           # slen= len(statsVals)
#             colorsTable = setRowColors(statsVals,"#b3f0ff","#33d6ff","pink",header_list)

#             window['-TABLE-'].update(values=statsVals,row_colors=colorsTable)
        return statsVals,colSortState


#######################################################################
def dfAnalyze(df):
    stats = {}
    
    for col in df.columns:
        typs = df[col].apply(type).value_counts().to_dict()
        stats[col] = {}
        if str in typs:
           stats[col]["string"] = typs[str]
        else:
           stats[col]["string"] = 0

        if int in typs:
           stats[col]["integer"] = typs[int]
        else:
           stats[col]["integer"] = 0

        if float in typs:
           stats[col]["float"] = typs[float]
        else:
           stats[col]["float"] = 0

        if bool in typs:
           stats[col]["boolean"] = typs[bool]
        else:
           stats[col]["boolean"] = 0

        stats[col]["Missing"] = df[col].isna().sum()
        stats[col]["% Missing"] = round(df[col].isna().sum()/df.shape[0]*100,1)
        
    return stats

############################################# 

def getValues(stats,header):

    statsVals=[]
    for col in sorted(stats.keys()):
         vals=[]
         vals.append(col)
         for k in header[1:]:
            vals.append(stats[col][k.strip()])
         statsVals.append(vals)
    return statsVals     
############################################################

def getFilesClicked(file,window):
#         if len(values["-FILE1-"]) > 0:
#             file = values["-FILE1-"]
#         elif len(values["-WEB1-"]) > 0:
#             file = values["-WEB1-"]
        
        df = getFile("Local",file,window)
        stats = dfAnalyze(df)

        return df,stats

############################################################

def getRowClicked(table,columns):
    col=""
    e = table.user_bind_event 
    region = table.Widget.identify('region', e.x, e.y)
    if region == 'heading':
        row = 0
    elif region == 'cell':
        row = int(table.Widget.identify_row(e.y))  
        col = columns[row-1][0]
    return row,col    

#####################################################################

def showDFUN(col,unq,windowParent,colr1,colr2):
    print("SHOWING BABY ")
    valuesUNQ = list(zip(unq.index.tolist(),unq.tolist()))
    hUNQ = []
    hUNQ.append("Values")
    hUNQ.append("Count")

    
    sortState = {}
    for val in hUNQ:
        sortState[val]=-1
    layout2 = [    
                   [sg.Text(f"Showing unique Values for Column"),sg.Text(f" {col}",text_color="red"),sg.Text(f" and Reg Ex",text_color="black")],
                
                   [sg.Button('Quit')],
            
                   [sg.Button('Write Unique'),
                    sg.Table(values=valuesUNQ,
                       background_color=colr1,vertical_scroll_only=False,col_widths=60,font='Courier 10 bold ' ,
                       auto_size_columns=True,enable_events=True,def_col_width=25,text_color="black",
                       justification='right',alternating_row_color=colr2,
                       key='-TABLE-', headings = hUNQ,metadata=sortState)]
            ]
    window2 = sg.Window(f"Unique", layout2,finalize=True,resizable=True, grab_anywhere=False)

    table = window2['-TABLE-']
    table.bind('<Button-1>', "Click")
    return window2,table,valuesUNQ,hUNQ


###########################################################

def sortUniqe(table,window,dataStore,headerStore):
    try:
        e = table.user_bind_event
        region = table.Widget.identify('region', e.x, e.y)
        sortAsc = {}
        sortAsc[1] =False
        sortAsc[-1]=True
   #     print("R ",region)
        if region == 'heading':
            values = dataStore[table]
           
            header = headerStore[table]
           
            column = int(table.Widget.identify_column(e.x)[1:])
            col=header[column-1]
          
            sortState = table.metadata
            sortState[col]*=-1
            table.metadata = sortState
          
            values = sorted(values, key=lambda element: (element[column-1]),reverse=sortAsc[sortState[col]]) 
           
            window["-TABLE-"].update(values=values)
            dataStore[table] = values
    except Exception as err:
        exceptionLog(err,inspect.currentframe().f_code.co_name)
                    
        

def wrap(string, lenght=60):
    if isinstance(string,str):
       return '\n'.join(textwrap.wrap(string, lenght))
    else:
        return ""
def runETL(k):
    global extract,a,string
    text2 = ""
  #  print(k)
## Build Summary Text string
    for ky1 in dataSetsEtl[k].keys():
        text2+=f"{ky1}:\n" 
        for k2,v2 in dataSetsEtl[k][ky1].items(): 
            sl = 10 - len(k2)
            s = " "*sl
        
            if isinstance(v2,list) == False:
                text2+=f"    {k2:10s}{s} :  {v2}\n"
            else:
                text2+=f"    {k2:10s}{s} : {v2[0]}\n"
                if len(v2) > 1:
                    for val in v2[1:]:
                        text2+= f"                   {s} : {val}\n"

        text2+=f"\n\n"
## Build actions
    extract = ""
    if 'extract' in dataSetsEtl[ds]:
        if (dataSetsEtl[ds]['extract']['language'] == 'node'):
            pgm =  dataSetsEtl[ds]['extract']['file']
            options=""
            if 'options' in dataSetsEtl[ds]['extract']:

                for opts in dataSetsEtl[ds]['extract']['options']:
                    options+= f" {opts}" 
            extract = f"node /home/joe/bic_etl/{pgm} {options}"
        #    print(extract)
    text2+=f"\n\nActions Strings\nExtract\n{extract}"
        
    return text2,extract
    
    
def cronGUI():
    global ds,text2,a,string,numWindows,color1,color2,file
    dataStore = {}
    datasets = init()
    print("LL ",len(dataSetsEtl))
    mf = pd.read_excel("BICDataInventoryandMetadata.xlsx",skiprows=0,sheet_name="Inventory_Active",engine="openpyxl")
    
    mf["CIM Link"] = mf.apply(map4x4,axis=1)

    mf=mf.loc[~mf["Standardized Title for Dataset"].isna()]
    mfShort = mf[['Standardized Title for Dataset', 'Data Link','CIM Data Type','GoCodePublishYear','Standardized Short Description','CIM Link']]
    mfShort["Standardized Title for Dataset"] = mfShort["Standardized Title for Dataset"].astype(str)   
    
    header_list = list(mfShort.columns)
    # mfV = []
    # a = ["","",nan,nan,""]
    # mfV.append(a)
    layout = [
             [sg.Text("BIC Cron DataSets")],
             [sg.Combo(datasets,enable_events=True,key="-DATASET-",font='Courier 10 bold ')],
              [sg.Button('Quit'),sg.Button("Plot Crons")],
             [sg.Button("GO")],
              [sg.Button("Extract",font='Courier 10 bold ',key="-EXTRACT-",visible=False),
               sg.Button("Analyze-Extr",font='Courier 10 bold ',key="-EXTRACTANAL-",visible=False),
               sg.Text("",visible=False,font='Courier 10 bold ',key="-EXTRACTINFO-")],
              [sg.Button('Go to CIM Dataset',visible=False,key="-LINK-",metadata="")],
              [sg.Text(text="Dataset Summary",enable_events=True,font='Courier 15 bold ',background_color="blue",key="-OUTPUT-",size=[70,10]),
       #       [sg.Multiline(default_text="Dataset Summary",key="-OUTPUT-",size=[70,10]),

               sg.Multiline(default_text="ETL Summary",key="-ETL-",size=[70,20],font='Courier 15 bold ')
              ]

       ]
    # Create the Window
    sg.theme('Dark Green 5')
    window2 = sg.Window('ETL', layout,finalize=True,resizable=True)
    #window = sg.Window('Window Title', layout, web_port=2222, web_start_browser=False)
    #table = window2['-TABLE2-']

    #window2.move(window.current_location()[0]+600, window.current_location()[1])
    try: 
        while True:

         #   event, values = window2.read()
            wid, event, values = sg.read_all_windows()
            print('event: ',event, 'values: ',values)
 #           print("W2",event,values)
        #    window, event, values = sg.read_all_windows()
            if event == sg.WIN_CLOSED or event == 'Quit':
                window2.close()
        #        sys.exit(1)
                what = "QUIT"
                break
           # elif event == "-DATASET-":
            elif event == "GO":#

                
                (ds)  = list(values.items())
                ds=ds[0][1]
                ds = "Paid Solicitors Disclosed on Charity Registration Forms in Colorado"
           #     print("DS ",ds)
                mmf = mfShort.loc[mfShort["Standardized Title for Dataset"].str.strip() == ds.strip()]
                if mmf.shape[0] > 0:
                #            mmf["Standardized Short Description"] = mmf["Standardized Short Description"].map(wrap)          
                    text = f"Title             : {mmf['Standardized Title for Dataset'].values.tolist()[0]}\nSocrata        : {mmf['Data Link'].values.tolist()[0]}\nData Type    : {mmf['CIM Data Type'].values.tolist()[0]}\nPublish Year: {mmf['GoCodePublishYear'].values.tolist()[0]}\nCIM Link : {mmf['CIM Link'].values.tolist()[0]}\nDescription  : {mmf['Standardized Short Description'].values.tolist()[0]}"            
                   # display(mmf)
                else:
                    text = "None"
                if ds in dataSetsEtl:
                      text2,extract = runETL(ds)
                else:
                      text2=""
                      extract = ""
                if len(extract) > 10:
                    window2["-EXTRACT-"].update(visible=True)
          #      print("T2",text2)
                window2["-OUTPUT-"].update(text)
                window2["-ETL-"].update(text2)
                link =  mmf['CIM Link'].values.tolist()[0]
           #     print("Link ",mmf['CIM Link'],link.find("http"))
                
                if link.find("http") > -1:
                     window2["-LINK-"].update (visible=True)
                else:
                     window2["-LINK-"].update (visible=False)
               
            elif event == "-LINK-":
                    print("Going To: ",link) 
                    if len(link) > 10:
                        webbrowser.open(link)
            elif event == "Plot Crons":    
                plotCrons(crons_all)
                
            elif event == "-EXTRACT-":
                a=subprocess.run(extract,shell=True,capture_output=True,text=True)
                string= "\n\n-------------------------------------\n"
                string+= "Extract Output\n"
                if len(a.stderr) > 0:
                    string+= "fError:\n {a.stderr}"
                string+=f"STDOUT: {a.stdout}"
                window2["-ETL-"].update(string,append=True)
                sx = string.find("successfully download to")
                string[sx+25:]
                sxo = string[sx+25:].find("!")
              #  print(sxo)
                file=f"/home/joe/bic_etl{string[sx+25:sx+25+sxo]}"
                
               # print(file)
                if os.path.isfile(file):
                    print(f"{file} FOUND FOUND FOUND")
                    window2["-EXTRACTANAL-"].update(visible=True)
                    fileStats = os.stat(file)
                    dt = datetime.fromtimestamp(fileStats.st_ctime)
                    string = f"{fileStats.st_size:,} bytes Created: {dt}  file: {file}"
                    window2["-EXTRACTINFO-"].update(string,visible=True)
                   
            
           
            elif event == "-EXTRACTANAL-":
                file=stringArgs(extract)
                print("FILE ",file)
                df1,stats1 = getFilesClicked(file,window2)
                color1 = colorPairs[numWindows][0]
                color2 = colorPairs[numWindows][1]
                
                numWindows+=1
                if numWindows > len(colorPairs):
                    numWindows=0
                    
                WindowC,tabl1,valsFile1,rowFile1Colors = compareWindow(stats1,df1,file)
                dataStore[tabl1]=valsFile1
                windowsOpen["main"].append(WindowC)
            
            ###########
            elif event == "-TABLEFILE-Click":
               
                row,col=getRowClicked(tabl1,valsFile1)
                print("R,C ",row,col)
                if row == 0:
                   print("HEADER BABY",col)
                   valsFile1,columnSortStateTable1 = sortTable(row,stats1,tabl1,event,header_list)          
        #           setRowColors(valsFile1,"#b3f0ff","#33d6ff","pink",header_list)
                   wid["-TABLEFILE-"].update(values=valsFile1,row_colors=rowFile1Colors)
                   wid["-TABLEFILE-"].metadata=columnSortStateTable1
                else:
                    unq = df1[col].value_counts()
                    w,t,values,uHead = showDFUN(col,unq,wid,"#b3f0ff","#33d6ff")
                    dataStore[t] = values
                    headerStore[t] = uHead
                    
            elif event == "-TABLE-Click":
                print("TC ",wid)
                table = wid['-TABLE-']
                print("TC ")
                
                sortUniqe(table,wid,dataStore,headerStore)
                
    except  Exception as err: 
       exceptionLog(err,inspect.currentframe().f_code.co_name)
 
for wid in windowsOpen["main"]:
    print("window MA",wid)
    if wid:
        print("cosing ",wid)
        wid.close()
        wid = None

cronGUI()



26 run_etl.json files found
LL  98


/tmp/ipykernel_2914/1163233529.py:492: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mfShort["Standardized Title for Dataset"] = mfShort["Standardized Title for Dataset"].astype(str)


event:  GO values:  {'-DATASET-': '', '-ETL-': 'ETL Summary'}
event:  -EXTRACT- values:  {'-DATASET-': '', '-ETL-': 'extract:\n    language     :  node\n    file             :  general/scripts/sftp_extract.js\n    options       : -f charity/char_orgs_sol.txt\n                       : -o cdos/business/nonprofit/data_source/\n                       : -a .tsv\n\n\ntransform:\n    language     :  node\n    file             :  scripts/char_orgs_sol.js\n\n\nload:\n    file             :  char_paid_solicitors.sij\n    type             :  datasync\n    format         :  csv\n    load_file   :  char_orgs_sol.csv\n\n\n\n\nActions Strings\nExtract\nnode /home/joe/bic_etl/general/scripts/sftp_extract.js  -f charity/char_orgs_sol.txt -o cdos/business/nonprofit/data_source/ -a .tsv'}
/home/joe/bic_etl/cdos/business/nonprofit/data_source/char_orgs_sol.tsv FOUND FOUND FOUND
event:  Quit values:  {'-DATASET-': '', '-ETL-': 'extract:\n    language     :  node\n    file             :  general/scripts/sft

In [ ]:
fileStats

In [ ]:

timestamp = datetime.datetime.fromtimestamp(fileStats.st_ctime)
print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
timestamp = datetime.datetime.fromtimestamp(fileStats.st_atime)
print(timestamp.strftime('%Y-%m-%d %H:%M:%S'))